In [16]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from dotenv import load_dotenv
import pathlib
# from tqdm import tqdm
from tqdm.autonotebook import tqdm
from pathlib import Path
from pprint import pprint
import numpy as np
import hashlib

import polars as pl
from glob import glob

from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

from google import genai
from google.genai import types
from langchain_openai import ChatOpenAI

from PIL import Image

root_dir = Path(os.getcwd()).parent.parent
sys.path.insert(0, str(root_dir))

pl.Config.set_fmt_str_lengths(300)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(20);

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from src.d01_data.data import (read_json, create_index_if_not_exists, json_dump, pdf_to_images)
from src.d03_modeling.modeling import get_image_rotation, get_doc_id_from_image, gemini_translate_and_format, openai_translate_and_format
from src.d00_utils.utils import get_doc_id_data, campos_to_srt, parse_campos_a_rellenar

load_dotenv('../../.env')
 
raw_path = root_dir / 'data' / '01_raw'
intermediate_path = root_dir / 'data' / '02_intermediate'
clean_path = root_dir / 'data' / '03_clean'
output_path = root_dir / 'data' / '04_model_output'

In [18]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_MODEL = os.getenv('OPENAI_MODEL')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
GEMINI_MODEL = os.getenv('GEMINI_MODEL')

client = genai.Client(api_key=GEMINI_API_KEY)

In [19]:
fotos = sorted(glob(str(raw_path / 'fotos' / 'EX-10' / '*.jpg')))
foto = fotos[1]

idioma = 'frances'

In [13]:
rotation = get_image_rotation(client=client, model=GEMINI_MODEL, image_path=foto)
foto = Image.open(foto).rotate(rotation)

Selected rotation: -90º


In [6]:
doc_id = get_doc_id_from_image(client=client, model=GEMINI_MODEL, image=foto)

In [ ]:
doc_id_data = get_doc_id_data(doc_id=doc_id.codigo_documento, doc_folder=clean_path / 'formularios')
del doc_id_data['Código del documento']

In [78]:
traduccion = gemini_translate_and_format(client=client, model=GEMINI_MODEL, language=idioma, texto_a_traducir=doc_id_data['Secciones a rellenar del formulario'])
traduccion = traduccion.parsed.traduccion_formateada

In [82]:
texts_to_translate = [(idioma, doc_id_data[k]) for k in doc_id_data]
traducciones = await openai_translate_and_format(documents=texts_to_translate, model=OPENAI_MODEL)